In [ ]:
#set up
import requests
import re
import csv
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
#use beautiful soup to extract all states listed on 2020 Census - School District Reference Map website
census_state_url = 'https://www2.census.gov/geo/maps/DC2020/PL20/'
census_state_html_text = requests.get(census_state_url).text
soup = BeautifulSoup(census_state_html_text, 'html.parser')
states_code = []
for link in soup.find_all('a'):
    href = link.get('href')
    pattern = "^st\d+_.{2}/$"
    found = re.search(pattern, str(href))
    if found:
        states_code.append(href[:-1])
states_code = states_code[:51]

In [ ]:
#extract all states info listed on 2020 Census - School District Reference Map website
records = list()
for each_state in states_code:
    print ("searching for " + each_state)
    census_each_state_url = census_state_url+each_state+'/schooldistrict_maps/'
    census_each_state_html_text = requests.get(census_each_state_url).text
    each_state_soup = BeautifulSoup(census_each_state_html_text, 'html.parser')
    county_code = []
    for link in each_state_soup.find_all('a'):
        href = link.get('href')
        pattern = "^c\d+_\w+-?\w+"
        found = re.search(pattern, str(href))
        if found:
            county_code.append(href[:-1])        
    for each_county in county_code:
            #print ("searching for " + each_county)
            each_county_name_list = each_county.split('_')
            each_county_name = ''
            each_county_code = each_county_name_list[0]
            for i in each_county_name_list[1:]:
                each_county_name = each_county_name + ' ' + i
            census_each_county_url = census_each_state_url + each_county + '/DC20SD_' + each_county.split('_')[0].replace('c', 'C')+'_SD2MS.txt'
            census_each_county_html_text = requests.get(census_each_county_url).text
            each_county_soup = BeautifulSoup(census_each_county_html_text, 'html.parser')
            county_text = each_county_soup.text.split('\n')
            school_district_list = []
            for item in county_text:
                item_list = item.split(";")
                if item_list[0] == "Unified":
                    school_district_list.append(item_list[-3])
            for school_district in school_district_list:
                records.append({ 
                                'State': each_state.split('_')[1].upper(), 
                                'County_name':each_county_name.capitalize(),
                                'County_code':each_county_code,
                                'School_district': school_district
                                })

In [ ]:
#save dict as pd
school_df = pd.DataFrame.from_dict(records)

In [ ]:
#convert df to csv
school_df.to_csv('final_school_district.csv')